In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('rfm.tsv') as file:
    array = [row.strip() for row in file]

In [3]:
df = pd.DataFrame(array, columns=['data'])

In [4]:
data = df['data'].str.split(pat = '\t',expand=True)

In [5]:
data.rename(columns={0: 'timestamp', 1: 'user_id', 2:'url', 3:'territory_id', 4:'category_id', 5:'price'}, inplace=True)

In [6]:
data.head()

,timestamp,user_id,url,territory_id,category_id,price
0,1380052543,00002ad98ae4290672431d702c211b4a,/_c/YwrVrj_29IVZDmKxglT53wo7Pk8.html,10000,None,None
1,1375703582,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=y5fc6079b758f5a4e3646a8...,10000,None,None
2,1374479873,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,None,None
3,1374479892,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,None,None
4,1374480337,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,None,None


In [7]:
geo_table = pd.read_csv('geo_table', delimiter='\t')
cat_tree = pd.read_csv('categories_tree', delimiter='\t')

In [8]:
geo_table.head()

,id,name,parent_region_id,type,TZ_OFFSET
0,1,Москва и Московская область,3.0,5,10800
1,2,Санкт-Петербург,10174.0,6,10800
2,3,Центральный федеральный округ,225.0,4,10800
3,4,Белгород,120914.0,6,10800
4,5,Иваново,120943.0,6,10800


In [9]:
cat_tree.head()

,id,parent_id,hyper_cat_id,category_name,category_pass
0,984,978,91768,Этикет-пистолеты,\tОборудование\tОборудование для магазинов\tЭт...
1,23187,978,10469630,Рекламные дисплеи и интерактивные панели,\tОборудование\tОборудование для магазинов\tРе...
2,10411,978,5057803,Манекены,\tОборудование\tОборудование для магазинов\tМа...
3,19545,978,6509282,Терминалы сбора данных,\tОборудование\tОборудование для магазинов\tТе...
4,981,978,91765,Весы,\tОборудование\tОборудование для магазинов\tВе...


In [10]:
# Разобьем клиентов на тех, у кого есть платные клики и у кого их нет:
clients_pay = list(data[data['url'] == 'click']['user_id'].unique())

d_pay = data[data['user_id'].isin(clients_pay)]
d_unpay = data.drop(d_pay.index)

##### Для клиентов без платных кликов расчитаем величины R (давность последнего клика) и R ()

In [11]:
d_unpay['timestamp'] = d_unpay['timestamp'].astype('float', unplace = True)
d_unpay_end = d_unpay['timestamp'].max()

In [12]:
RFTable = d_unpay.groupby('user_id').agg({'timestamp': lambda x: (d_unpay_end - x.max()), 
                                        'user_id': lambda x: len(x)})

In [13]:
RFTable.head()

,timestamp,user_id
user_id,,
00002ad98ae4290672431d702c211b4a,8467067.0,1
00003cb9076039f584a12c2036fd3602,2289814.0,19
000065938a8c02438551bef260a5ded0,24547954.0,7
000098543e00d361552ab956ec525772,16390403.0,1
0000d4b93e501f038977d4ae8fb7693b,5285039.0,2


In [14]:
# Вычисление квантилей для сегментирования
R1 = RFTable['timestamp'].quantile(0.2)
R2 = RFTable['timestamp'].quantile(0.8)

F1 = RFTable['user_id'].quantile(0.2)
F2 = RFTable['user_id'].quantile(0.8)

In [15]:
RFTable.loc[RFTable['timestamp'] < R1, 'R_cat'] = 1
RFTable.loc[(R1 <= RFTable['timestamp']) &  (RFTable['timestamp']< R2), 'R_cat'] = 2
RFTable.loc[RFTable['timestamp'] >= R2, 'R_cat'] = 3

RFTable.loc[RFTable['user_id'] <= 2, 'F_cat'] = 1
RFTable.loc[(3 <= RFTable['user_id']) & (RFTable['user_id'] < 8), 'F_cat'] = 2
RFTable.loc[RFTable['user_id'] >= 8, 'F_cat'] = 3

In [16]:
RFTable['R_cat'] = RFTable['R_cat'].astype('int64', inplace = True)
RFTable['F_cat'] = RFTable['F_cat'].astype('int64', inplace = True)

RFTable['R_cat'] = RFTable['R_cat'].astype('str', inplace = True)
RFTable['F_cat'] = RFTable['F_cat'].astype('str', inplace = True)

In [17]:
RFTable['RF_segment'] = RFTable['R_cat'] + RFTable['F_cat']

In [18]:
# Получившиеся сегменты (клиенты без платных кликов)
RFTable['RF_segment'].value_counts()*100/len(RFTable)

21    28.483055
23    16.620274
22    14.896426
31    10.980974
12     7.412752
13     6.960985
11     5.626181
32     4.661564
33     4.357790
Name: RF_segment, dtype: float64

##### Считаем RFM отдельно для клиентов с платными кликами

In [19]:
d_pay['price'].fillna(0, inplace = True)

C:\Users\mi\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [20]:
d_pay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7358563 entries, 56 to 10007121
Data columns (total 6 columns):
timestamp       object
user_id         object
url             object
territory_id    object
category_id     object
price           object
dtypes: object(6)
memory usage: 393.0+ MB


In [21]:
d_pay['timestamp'] = d_pay['timestamp'].astype('float64', copy = False)
d_pay['price'] = d_pay['price'].astype('int', copy = False)

C:\Users\mi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
RFMTable_pay = d_pay.groupby('user_id').agg({'timestamp': lambda x: (d_unpay_end - x.max()), 
                                        'user_id': lambda x: len(x), 'price': lambda x: sum(x)})

In [23]:
R11 = RFMTable_pay['timestamp'].quantile(0.2)
R22 = RFMTable_pay['timestamp'].quantile(0.8)

F11 = RFMTable_pay['user_id'].quantile(0.2)
F22 = RFMTable_pay['user_id'].quantile(0.8)

M11 = RFMTable_pay['price'].quantile(0.2)
M22 = RFMTable_pay['price'].quantile(0.8)

In [24]:
RFMTable_pay.loc[RFMTable_pay['timestamp'] < R11, 'R_cat'] = 1
RFMTable_pay.loc[(R11 <= RFMTable_pay['timestamp']) &  (RFMTable_pay['timestamp']< R22), 'R_cat'] = 2
RFMTable_pay.loc[RFMTable_pay['timestamp'] >= R22, 'R_cat'] = 3

RFMTable_pay.loc[RFMTable_pay['user_id'] <= F11, 'F_cat'] = 1
RFMTable_pay.loc[(F11 <= RFMTable_pay['user_id']) & (RFMTable_pay['user_id'] < F22), 'F_cat'] = 2
RFMTable_pay.loc[RFMTable_pay['user_id'] >= 8, 'F_cat'] = 3

RFMTable_pay.loc[RFMTable_pay['user_id'] <= M11, 'M_cat'] = 1
RFMTable_pay.loc[(M11 <= RFMTable_pay['user_id']) & (RFMTable_pay['user_id'] < M22), 'M_cat'] = 2
RFMTable_pay.loc[RFMTable_pay['user_id'] >= M22, 'M_cat'] = 3

In [25]:
RFMTable_pay['R_cat'] = RFMTable_pay['R_cat'].astype('int64', inplace = True)
RFMTable_pay['F_cat'] = RFMTable_pay['F_cat'].astype('int64', inplace = True)
RFMTable_pay['M_cat'] = RFMTable_pay['M_cat'].astype('int64', inplace = True)

RFMTable_pay['R_cat'] = RFMTable_pay['R_cat'].astype('str', inplace = True)
RFMTable_pay['F_cat'] = RFMTable_pay['F_cat'].astype('str', inplace = True)
RFMTable_pay['M_cat'] = RFMTable_pay['M_cat'].astype('str', inplace = True)

In [26]:
RFMTable_pay['RFM_segment'] = RFMTable_pay['R_cat'] + RFMTable_pay['F_cat'] + RFMTable_pay['M_cat']

In [27]:
# Получившиеся сегменты для клиентов с платными кликами
RFMTable_pay['RFM_segment'].value_counts()/55277

232    0.371312
332    0.116631
132    0.112289
211    0.099408
233    0.075221
231    0.054037
133    0.051341
311    0.047524
331    0.022577
111    0.021890
131    0.014491
333    0.013279
Name: RFM_segment, dtype: float64

In [28]:
RFMTable_pay.head()

,timestamp,user_id,price,R_cat,F_cat,M_cat,RFM_segment
user_id,,,,,,,
00023fcc5c93cb4674bdad0d448272e6,2168761.0,390,425,2,3,3,233
00029bdb868e12bb62b8c574b274dbb7,31316648.0,102,90,3,3,2,332
00035d94fcd7e814d56531cf95b81ceb,12958857.0,9,159,2,3,1,231
00045ac92fb4ed78bc66c40f52775acd,5022312.0,15,50,2,3,2,232
00056ac58d63ef1d4b5fb73324bf845d,7958064.0,122,145,2,3,2,232


In [ ]:
# Самая частая категория (по территории) (на примере сегмента '333')
for i in d_pay[d_pay['user_id'].isin(RFMTable_pay[RFMTable_pay['RFM_segment'] == '333'].index)]['territory_id'].unique():
    print(i, d_pay[d_pay['territory_id'] == i]['category_id'].value_counts()[0])

10000 526505
2 34896
0 1050
213 151231
10752 665
10740 2423
56 2652
172 3093
62 3368
65 5253
75 597
35 11569
47 10667
10761 191
53 586
197 640
67 587
66 1288
43 6155
4 2140
11 2029
54 6142
38 2560
9 1284
21619 1475
23 1152
21 441
25 213
10928 466
17 37
39 8241
968 324
15 1867
76 1690
11115 96
16 2189
225 192
18 329
14 2242
10754 409
55 3138
20667 405
10727 35
21644 584
1 141
10723 116
30 540
49 940
21627 79
21745 126
11056 275
48 1016
114678 223
50 2427
11108 4
21414 35
87 48
973 668
12 824
11119 11
236 1545
240 2164
51 4021
21735 274
21623 240
10871 128
10664 606
45 1047
193 2170
28 1520
37 1468
192 1134
235 220
21620 247
11217 5
239 591
11139 251
10841 24
11391 3
10408 15
10747 596
21653 548
20032 57
191 1773
10951 799
237 334
102557 24
975 175
974 74
8 971
6 1907
36 1832
33 368
63 5003
11266 17
37180 119
20088 15
198 288
11162 28
11172 3
10661 74
10742 248
10870 47
11278 5
11067 198
10728 7
195 458
21621 674
37147 52
37126 6
194 2820
19 818
20255 15
11164 39
11036 58
10735 777
20 82

In [ ]:
# Самая частая категория, которую смотрел клиент (на примере сегмента '333')
for i in RFMTable_pay[RFMTable_pay['RFM_segment'] == '333'].index:
    print(i, d_pay[d_pay['user_id'] == i]['category_id'].value_counts()[0]) 

0101be95bedcda0a20be36f3db2db236 150
010309f722cabcf3b4dadfc6df5d9a55 225
010ec743d96745cc383864474c83d4d7 220
0123cd734cdebc44573cf7545f45d382 150
0189cf8da24c6e1e0eb30ae93a1ce7a1 218
019fab1996d6dd6b1c5e0e70adce185c 210
01a543051533cb7b1572476cf174fc01 381
021cc44fc7cebf6c3652f297514520ff 1426
021f77e43f6a3e20902068ea6613452e 45
02330e65f2be9a25e1d15fae087d4302 180
027a8f1126ff3d7b3bf3885e838ea3af 296
02968b90259ee69345824e1cc45cf2e4 84
02d811c724addf1ad9e2e15f07632ddd 378
032de6050568af5a8391f6ad214a48a8 528
03b3a55078e96f22781451b975fd21d4 181
0572ea4aa8d17b6e610712d6dcdaa81a 1090
05b9437ec4e5079d1c6001f3c399651c 154
060617b75b4c6f494bc7f02be62b7814 58
0609cf4c061278ab682064bd8dab3a82 61
06408ba9f848db9f86353688d5516242 143
0671b72103622d86eb710b6f65b51a32 46
0685ebc2089732471e258986ec99a39c 186
06adcf4d00384bdb09545503d7ff805b 181
071267b0556454c0449c9ae48c89ea22 187
07aceff23da157a149adcbd649401c6c 200
07bd51a1bf6afd4cfe10d51f86b9a761 296
081ba0a720352dcae7e155a9b49b9184 97
096c9

510cff7bbf34325911cb4e1862552762 183
5167dcbbb6014045af459294d423ef1a 309
5223746a4ad985f9463fd3fbb21644c6 130
5265ac85d8582576e18ba72fe07e3701 77
535c3c30b3365f22a775f92b8a51775e 104
53b7bdcb6e2d6c4f5e2fec938016b9e9 110
53bcefec56c8673010b35d6c72a34613 339
53eefbaaa21707ef42fefa9a4aed896f 521
542820819a9c851a92dd6b222aa0c069 88
5479819dd01e046ea9244e800d23da34 92
54d38d41ecf881ee59efa3bff9835174 89
55b3ad06b2d3631e8e79166499c962d2 74
55b96e2a555142473c3b6331cfc29830 117
55c6088081869d59eae2dc7e439dd38d 89
55d6d6a628def465b9d5e77b22692787 344
55ff79c4bf1f4a0483dd6bc04eb89d81 172
5639ca2fcac19b29ab4233f7e06b1ca2 211
571ceba50314102bfa659d4074ffdd0b 34
572f905b90bdffd61966053c03591fd4 308
57408017fd7409aefe30dc2c9db65a97 38
577b6596a7ca061c637ec308de603713 85
577f6b897d2d04eb9bcf256b2e31e4a2 50
5868efe26c0a9d9858e21222a9b25e81 125
58a84000463b6e0f082f5b5ec986162d 315
5905e837d584d521cf14eccdd5f304db 64
5947d6096945f52e7a0ef370b5af4a68 158
59a0ca6197e7db866683dece7d8e0ff8 111
59c090cf60d1

99e7d712c0c933e74e72a315ee82feeb 117
99ee2415eb74e0aeca95bcbd720e841a 79
9a8def881b5d4976ef6f8a5db5a7a918 79
9ab270debdab3d061764d07026bfa2ab 104
9b1e4174ecfc4632ce4a148f26125dd1 43
9b6c61a574e6e14305d6e6ea8f3b998e 279
9b8b2056e063cc47387a5390e8fe089b 336
9bb97bce85d5003bfcf3e9b25c48d1c9 137
9c104367cf206dd7d46cfd36a03ef8d1 95
9c1f6ae15dd5beb4d5e6a458ca7b8407 488
9ce79740e6e10833ed4d5523f789c5fc 150
9ceb2c9c1d1b979ac9742814afbb1c0e 225
9dbc9b9e660e7274d9219ed784e51ae7 80
9e7ad6b6e28b05eaca91777223aab9ab 196
9ec21b05aa0d599b5aac179c4c91dd08 348
a02bb7e4e13d7384bf53804f67b89468 59
a02e8373c3bac8794ba6388f6ee80477 438
a09efa3bb207795bb56767be19259b15 238
a10c8c31073848c0a33d1395ca9c14b7 207
a18ea6a106647e356d7b1547677f167a 832
a19c58762da3a6ce4142ab756be7ef40 91
a3111ffb66dcffc1d9c8e065bf4fb2f1 198
a31eca3a51a28a152e6857926fefe292 280
a347014b5696b874436402b358c1cde2 79
a3b68ca88d2d37b00eed380b0a3bab69 100
a4f9b3e28e6109a6aae28c506f8e27ab 115
a544544f0bb09354219c5077f30924ef 299
a581eff51

In [ ]:
#Самая частая категория, в определенном сегменте
for i in d_pay['territory_id'].unique():
    print(i, d_pay[d_pay['territory_id'] == i]['category_id'].value_counts()[0]) 